# RAG

* R --> Retrievel
* A --> Augented
* G --> Generation

* First the document is stored in vector database.
* We take prompt from the user.
* Relevant informatoion is **Retrived** from the Vector database on the basis the prompt.
* The prompt and the context from the retrieved document is **Augmented**.  
* This augmented prompt is used to **Generate** the response from LLM.

In [ ]:
# Requirements
# !pip install faiss-cpu langchain-community sentence-transformers

   ---------------------------------------- 0.0/18.9 MB ? eta -:--:--
   -- ------------------------------------- 1.0/18.9 MB 12.5 MB/s eta 0:00:02
   ----------- ---------------------------- 5.5/18.9 MB 17.6 MB/s eta 0:00:01
   ----------------------- ---------------- 11.0/18.9 MB 20.5 MB/s eta 0:00:01
   --------------------------------- ------ 16.0/18.9 MB 21.8 MB/s eta 0:00:01
   ---------------------------------------- 18.9/18.9 MB 20.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.5 MB ? eta -:--:--
   ---------------------------------------- 2.5/2.5 MB 22.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.0 MB ? eta -:--:--
   --------------- ------------------------ 4.7/12.0 MB 22.8 MB/s eta 0:00:01
   --------------------------------- ------ 10.0/12.0 MB 24.2 MB/s eta 0:00:01
   ---------------------------------------- 12.0/12.0 MB 23.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/566.1 kB ? eta -:--:--
   -----------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 6.0.7 requires ipython!=8.17.1,<9.0.0,>=8.13.0; python_version > "3.8", but you have ipython 9.5.0 which is incompatible.
streamlit 1.45.1 requires packaging<25,>=20, but you have packaging 25.0 which is incompatible.


In [9]:
# Import the libraries
import os
import google.generativeai as genai
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from pypdf import PdfReader
from langchain_text_splitters import RecursiveCharacterTextSplitter

import faiss
from langchain_community.vectorstores import FAISS

# Step 1 : Configure the model

In [2]:
# First lets configure the model

# LLM Model
gemini_key = os.getenv('Google_api_key1')
genai.configure(api_key=gemini_key)
model = genai.GenerativeModel('gemini-2.5-flash-lite')

# Configure Embedding Model
embedding_model = HuggingFaceBgeEmbeddings(model_name='all-MiniLM-L6-v2')

C:\Users\sarth\AppData\Local\Temp\ipykernel_7328\1639348513.py:9: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name='all-MiniLM-L6-v2')


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Users\sarth\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\sarth\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# Step 2: Get the document and extract the text

In [5]:
# Step 2: Lets read the pdf and extract the text from it

pdf_file = PdfReader('D:\GenAI\RAGBASED_CHATBOT\RagBasedChatBot\sample.pdf')

raw_text = ''
for page in pdf_file.pages:
    text = page.extract_text()
    if text:
        raw_text = raw_text + text + '\n'
''

''

# Step 3: Chunking

In [6]:
# First we need to split the text

splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
chunks = splitter.split_text(raw_text)

In [7]:
len(raw_text)

37306

In [8]:
len(chunks)

49

# Step 4: Create the Vector Database. (FAISS)

In [10]:
vector_store = FAISS.from_texts(chunks,embedding_model)

# Step 5: Get the prompt from the user

In [27]:
prompt = 'Give the 5 line summary of the project.'

# Step 6: Retrieval (R)

In [19]:
retriever = vector_store.as_retriever(search_kwargs={'k':3})
retrived_docs = retriever.invoke(prompt)

In [20]:
retrived_docs

[Document(id='96bd23ad-63ab-4822-8908-4148e83976fd', metadata={}, page_content='Dr.S.Vigneshwari M.E., Ph.D., and Dr.L.Lakshmanan M.E., Ph.D., Heads of the  \nDepartment of Computer Science and Engineering for providing me necessary \nsupport and details at the right time during the progressive reviews. \nI would like to express my sincere and deep sense of gratitude to my Project Guide  Dr. \nR. AROUL CANESSANE  M.E., Ph.D.,  for her valuable guidance, suggestions and  \nconstant encouragement paved way for the successful completion of my project work. \nI wish to express my thanks to all Teaching and Non -teaching staff members of the  \nDepartment of Computer Science and Engineering who were helpful in many \nways for the completion of the project.\n5  \n \n                                 TABLE OF CONTENT \n \nINDEX \nNO \n                                             TITLE PAGE \nNO \n1.                                 ABSTRACT 6 \n2. INTRODUCTION 7 \n3.                            

In [21]:
context = '\n'.join([d.page_content for d in retrived_docs])

In [22]:
print(context)

Dr.S.Vigneshwari M.E., Ph.D., and Dr.L.Lakshmanan M.E., Ph.D., Heads of the  
Department of Computer Science and Engineering for providing me necessary 
support and details at the right time during the progressive reviews. 
I would like to express my sincere and deep sense of gratitude to my Project Guide  Dr. 
R. AROUL CANESSANE  M.E., Ph.D.,  for her valuable guidance, suggestions and  
constant encouragement paved way for the successful completion of my project work. 
I wish to express my thanks to all Teaching and Non -teaching staff members of the  
Department of Computer Science and Engineering who were helpful in many 
ways for the completion of the project.
5  
 
                                 TABLE OF CONTENT 
 
INDEX 
NO 
                                             TITLE PAGE 
NO 
1.                                 ABSTRACT 6 
2. INTRODUCTION 7 
3.                                     AIM 13 
4.                                   SCOPE 13
documenting the artifacts of softwar

# Step 7: Augmenting (A)

In [28]:
augmented_prompt = f'''
<Role> You are helpful assistant using RAG.
<Goal> Answer the question asked by the user. Here is the question: {prompt}
<Context> Here are the documents retrieved from the vector database to support
the answer which you have to generate {context}
'''

# Step 8: Generation (G)

In [29]:
response = model.generate_content(augmented_prompt)
print(response.text)

The project, titled "A SYSTEMATIC APPROACH TOWARDS DESCRIPTION AND CLASSIFICATION OF CRIME INCIDENTS", focuses on using the Unified Modeling Language (UML) to develop object-oriented software. UML, a collection of best engineering practices, is crucial for modeling complex systems and software development processes, employing graphical notations for design expression. The project aims to provide an expressive visual modeling language for developing and exchanging meaningful models, with mechanisms for extensibility and specialization. It also emphasizes independence from specific programming languages and development processes. The core of the project involves input data preprocessing, feature extraction, and prediction/classification of crime types using a training dataset.
